In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-11-23 2021-11-29


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|█████████████████████▏                                                                                                              | 81/505 [00:54<04:26,  1.59it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|███████████████████████████████████████████████████████████████████████████████▍                                                   | 306/505 [03:25<02:23,  1.38it/s]

project_market prices documents must be a non-empty list


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 501/505 [05:37<00:02,  1.52it/s]

...

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [05:39<00:00,  1.60it/s]

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [05:40<00:00,  1.48it/s]

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████▉                                                                                                                   | 65/505 [02:06<14:07,  1.93s/it]

'date'


 16%|████████████████████▋                                                                                                               | 79/505 [02:33<13:32,  1.91s/it]

'date'


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [16:14<00:05,  1.91s/it]

...

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [16:17<00:01,  1.89s/it]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [16:19<00:00,  1.94s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [12]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
315,1.950237,-0.033598,-0.022363,0.192670,0.205713,MRNA,Moderna,Health Care
368,0.511355,0.257862,0.248402,0.057165,0.061112,PFE,Pfizer,Health Care
111,-0.115771,0.058899,0.066157,0.031805,0.036532,CLX,Clorox,Consumer Staples
390,0.297204,0.081298,0.051863,0.025887,0.036316,DGX,Quest Diagnostics,Health Care
366,0.260079,0.054666,0.038161,0.019319,0.034699,PKI,PerkinElmer,Health Care
431,-0.159475,0.070140,-0.076931,0.028804,0.032560,TTWO,Take-Two Interactive,Communication Services
133,0.454769,0.071580,0.046112,0.033299,0.026880,DHR,Danaher Corporation,Health Care
228,0.050739,0.063833,0.057100,0.033356,0.025631,HOLX,Hologic,Health Care
243,-0.209023,0.002492,-0.013987,0.071619,0.024573,INCY,Incyte,Health Care
78,0.365318,0.000000,0.000000,0.008613,0.022518,COG,Cabot Oil & Gas,Energy


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Health Care,0.171139,-0.012447,-0.034569,-0.010290,-0.008718
Consumer Staples,0.076774,0.023550,0.009808,-0.016458,-0.012720
Communication Services,0.109176,-0.074058,-0.056597,-0.019347,-0.016149
Materials,0.268012,0.059626,0.013593,-0.026945,-0.016690
Utilities,0.102479,0.029748,-0.012275,-0.019567,-0.017566
Information Technology,0.229123,0.040467,-0.006299,-0.018988,-0.024554
Industrials,0.258343,0.043397,-0.013822,-0.029753,-0.028903
Consumer Discretionary,0.273949,0.048094,-0.014814,-0.040210,-0.032460
Real Estate,0.432658,0.052814,-0.002199,-0.018047,-0.032578
